# key:
- h: alpha helix
- s: beta sheet
- \- : no SS
- ?: position not in 3ndy (reference protein)
- . :  position not in celA (query protein)

In [1]:
import sys
import pathlib
source_dir = (pathlib.Path.cwd()/'..')
sys.path.append(str(source_dir))

import dataloader

In [2]:
aln_path = '../../sequence_sets/aln_gh5.fasta'
aln_dataset = dataloader.MSADataset(aln_path)

celA = aln_dataset[0][0].numpy()
ndy3 = aln_dataset[1][0].numpy()

pdb_path = '../../sequence_sets/3ndy.pdb'
pdb_lines = open(pdb_path).read().strip().split('\n')

# atom = slice(13,16)
# chain = slice(16,17)
# aa = slice(17,20)
# poly = slice(21, 22)
# resi = slice(22,26)
# temp = slice(60,66)

# pdb_a = [l for l in pdb_atoms if l[poly]=='A' and (l[chain]==' ' or l[chain]=='A')]

a_alt = 16
a_chain = 21
pdb_atoms = [l for l in pdb_lines if l[0:4]=='ATOM' and l[a_chain]=='A' and (l[a_alt]==' ' or l[a_alt]=='A')]

h_chain = 19
pdb_helices = [l for l in pdb_lines if l[0:5]=='HELIX' and l[h_chain]=='A' ]

s_chain = 21
pdb_sheets = [l for l in pdb_lines if l[0:5]=='SHEET' and l[s_chain]=='A' ]

In [6]:
helix_start = slice(21,25)
helix_end = slice(33,37)
helix_pos = [(int(l[helix_start]), int(l[helix_end])+1, 'h') for l in pdb_helices]

sheet_start = slice(22,26)
sheet_end = slice(33,37)
sheet_pos = [(int(l[sheet_start]), int(l[sheet_end])+1, 's') for l in pdb_sheets]

In [10]:
all_pos = helix_pos+sheet_pos
all_pos = list(set(all_pos)) # removes duplicates
list.sort(all_pos, key = lambda pos: pos[0])
# print(*all_pos, sep='\n')

ndy3 # reminder
ss_labels = []
aa_count = 0
ss_i = 0
for aa in ndy3:
    if aa==20:
        ss_labels.append('?')
    else:
        ss_range = all_pos[ss_i]
        ss_start = ss_range[0]
        ss_end = ss_range[1]
        ss_type = ss_range[2]
        
        if aa_count < ss_start:
            ss_labels.append('-')
        elif aa_count < ss_end:
            ss_labels.append(ss_type)
        else:
            ss_i +=1
        
        aa_count += 1
        
# print(*zip(ss_labels, ndy3), sep='\t')

In [20]:
celA # reminder
celA_labels = []

for i, (aa,ss) in enumerate(zip(celA, ss_labels)):
    if not aa==20:
        celA_labels.append(ss)

celA_clean = [aa for aa in celA if not aa==20]

# print(*zip(celA_labels, celA_clean), sep='\n')

In [34]:
# label training set

from read_config import Config
import utils

config_path = 'configs/model_1000_15.yaml'
config = Config(config_path, root_dir = '../..')
training_set = utils.get_dataset_from_config(config, transform=None)

celA_aln = training_set[0][0]
ss_aln = []
i = 0
for aa in celA_aln:
    if aa == 20:
        ss_aln.append('.')
    else:
        ss_aln.append(celA_labels[i])
        i +=1

In [37]:
import pickle as pkl

data = {'query': celA_aln, 'ss': ss_aln}
pkl.dump(data, open('training_set_structure.pkl', 'wb'))